In [1]:
!pip install networkx

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install wiki_ru_wordnet

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install conllu

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import os
from io import open
from conllu import parse_incr
from conllu import parse
import nltk
nltk.download('wordnet')
import re
import pickle
import networkx as nx
from nltk.corpus import wordnet as wn
from matplotlib import pyplot as plt
%matplotlib inline

from wiki_ru_wordnet import WikiWordnet
ruwn = WikiWordnet()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
with open('vocab.pickle', 'rb') as f:
    lemmas_vocab = pickle.load(f)

In [59]:
def UDPipe_processing(sentimentwords):

    for i, filename in enumerate(os.listdir('parsed texts')):
        
        food_or_service_d = {}

        data_file = open('parsed texts/' + filename, "r", encoding="utf-8")

        for id_sent, tokenlist in enumerate(parse_incr(data_file)):

            for token in tokenlist:

                if token['lemma'] in sentimentwords:

                    if token['upostag'] == 'VERB':

                          for dependent in tokenlist:

                                if dependent['upostag'] != 'PUNCT'and dependent['upostag'] != 'NUM' and dependent['head'] == token['id']:

                                    food_or_service_d[dependent['lemma'], id_sent] = token['lemma'] + '\t' + filename + '\t' + str(id_sent+1) + '\t' + str(token['id']) + '\t' + str(sentimentwords[token['lemma']])

                    else:

                        for head in tokenlist:

                            if token['head'] == head['id']:

                                food_or_service_d[head['lemma'], id_sent] = token['lemma'] + '\t' + filename + '\t'+  str(id_sent+1) + '\t' + str(token['id']) + '\t' + str(sentimentwords[token['lemma']])

        #print(i)
        massiv = food_or_service(food_or_service_d)
    

In [60]:
def food_or_service(input_fs):
    
    a = set()
    
    dict_food = ['еда','блюдо','кухня','салат','суп','пища']
    dict_service = ['обслуживание','официант','официант','персонал','работник','сотрудник']
    
    for key, value in input_fs.items():
        key = key[0]
        
        if key in dict_food:
            res = 'Food'
            make_tags_txt(value, res, a)
        
        elif key in dict_service:
            res = 'Service' 
            make_tags_txt(value, res, a)
        
        else:
            for i in range(1):
                try:
                    s = ruwn.get_synsets(key)[i]
                    for w in s.get_words():
                        for key_word in dict_food:
                              if key_word in w.definition():
                                res = 'Food'
                                make_tags_txt(value, res, a)
                    pass

                    for key_word in dict_service:
                          if key_word in w.definition():
                            res = 'Service'
                            make_tags_txt(value, res, a)
                
                except(IndexError):
                      pass
    unique({'7\t31\tFood\t0\n', '6\t9\tFood\t0\n', '6\t2\tFood\t0\n', '8\t10\tFood\t0\n'})
    print(a)

In [61]:
def unique(set_a):
    
    for i in set_a:
        print(i)
        i = i.split('\t')
        for comp in set_a:
            if i[0] == comp.split('\t')[0] and i[2] == comp.split('\t')[2] and i[3] == comp.split('\t')[3]:
                print(i[0], comp.split('\t')[0])
                #new_elem = 

In [62]:
def make_tags_txt(text, res, array):
    
    meta_data = text.split('\t')
    
    lemma = meta_data[0]
    id_doc = meta_data[1].split('.')[0]
    id_sent = meta_data[2]
    id_token = meta_data[3]
    ton = meta_data[4]
    
    t = id_sent + '\t' + id_token + '\t' + res + '\t' + ton + '\n'
    
    array.add(t)
    
    return array
    
#     file_tags = open('texts tagged/{}_tags.txt'.format(id_doc), 'a')
    
#     file_tags.write(id_sent + '\t' + id_token + '\t' + res + '\t' + ton + '\n')
#     file_tags.close()
    
#     if res == 'Food':
        
#         food_tags = open('food_tags.txt', 'a')
#         food_tags.write(lemma+ '\t' + res + '\t' + ton + '\n')
#         food_tags.close()
        
#     elif res == 'Service':
        
#         service_tags = open('service_tags.txt', 'a')
#         service_tags.write(lemma + '\t' + res + '\t' + ton + '\n')
#         service_tags.close()

In [63]:
#os.mkdir('texts tagged')
m = UDPipe_processing(lemmas_vocab)

6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Food	0

6 6
6 6
8	10	Food	0

8 8
{'25\t5\tFood\t1\n', '54\t11\tService\t0\n', '64\t17\tFood\t1\n', '20\t13\tFood\t0\n', '57\t10\tService\t0\n', '58\t5\tFood\t1\n', '59\t9\tFood\t1\n', '69\t4\tFood\t0\n', '5\t20\tFood\t0\n', '41\t10\tFood\t1\n', '5\t20\tService\t0\n', '68\t7\tFood\t0\n'}
6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Food	0

6 6
6 6
8	10	Food	0

8 8
{'7\t31\tFood\t0\n', '6\t9\tFood\t0\n', '6\t2\tFood\t0\n', '8\t10\tFood\t0\n'}
6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Food	0

6 6
6 6
8	10	Food	0

8 8
{'4\t8\tFood\t0\n', '18\t11\tFood\t0\n', '10\t7\tFood\t0\n', '7\t4\tFood\t1\n', '7\t4\tService\t1\n', '3\t9\tFood\t1\n'}
6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Food	0

6 6
6 6
8	10	Food	0

8 8
set()
6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Food	0

6 6
6 6
8	10	Food	0

8 8
{'3\t47\tFood\t1\n', '4\t397\tFood\t1\n', '4\t404\tFood\t0\n', '4\t4\tService\t0\n', '4\t279\tFood\t1\n', '5\t32\tFood\t1\n'}
6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Fo

6	2	Food	0

6 6
6 6
7	31	Food	0

7 7
6	9	Food	0

6 6
6 6
8	10	Food	0

8 8
{'12\t5\tService\t1\n', '12\t5\tFood\t1\n', '1\t4\tFood\t0\n', '12\t12\tFood\t1\n', '7\t18\tService\t0\n', '1\t6\tFood\t1\n', '10\t14\tFood\t0\n', '5\t7\tFood\t1\n', '6\t8\tService\t1\n'}


In [23]:
print(m)

None


In [ ]:
for filename in os.listdir('parsed texts'):
    